In [2]:
import pandas as pd
import os

In [3]:
# Folder that house the Tourism data
data_folder = '/Users/mac/Documents/python/Tourism-Analysis/Tourism Dataset'

In [4]:
for f in os.listdir(data_folder):
    print(f)

Country.xlsx
Type.xlsx
Region.xlsx
Mode.xlsx
Continent.xlsx
User.xlsx
City.xlsx
Item.xlsx
Transaction.xlsx


In [5]:
country_df = pd.read_excel(os.path.join(data_folder, 'Country.xlsx'))
country_df.head()

,CountryId,Country,RegionId
0,0,-,0
1,1,Cameroon,1
2,2,Chad,1
3,3,Rwanda,1
4,4,Ethiopia,2


In [6]:
transaction_df = pd.read_excel(os.path.join(data_folder, 'Transaction.xlsx'))
transaction_df.head()

,TransactionId,UserId,VisitYear,VisitMonth,VisitMode,AttractionId,Rating
0,3,70456,2022,10,2,640,5
1,8,7567,2022,10,4,640,5
2,9,79069,2022,10,3,640,5
3,10,31019,2022,10,3,640,3
4,15,43611,2022,10,2,640,3


In [7]:
dfs = {}

for file in os.listdir(data_folder):
    if file.endswith('.xlsx'):
        sheet_name = file.replace('.xlsx', '')
        dfs[sheet_name] = pd.read_excel(os.path.join(data_folder, file))

In [8]:
dfs['Transaction'].head()

,TransactionId,UserId,VisitYear,VisitMonth,VisitMode,AttractionId,Rating
0,3,70456,2022,10,2,640,5
1,8,7567,2022,10,4,640,5
2,9,79069,2022,10,3,640,5
3,10,31019,2022,10,3,640,3
4,15,43611,2022,10,2,640,3


In [34]:
transaction_df = dfs['Transaction']
country_df = dfs['Country']
region_df = dfs['Region']
continent_df = dfs['Continent']
city_df = dfs['City']
item_df = dfs['Item']
type_df = dfs['Type']
user_df = dfs['User']
mode_df = dfs['Mode']

In [15]:
# Merge user_df with continent_df to get continent name
user_continent = user_df.merge(continent_df, on="ContenentId", how="left")

In [18]:
# Merge user_continent to region_df to get region name
user_region = user_continent.merge(region_df, on="RegionId", how="left")

In [23]:
city_df.head()

,CityId,CityName,CountryId
0,0,-,0
1,1,Douala,1
2,2,South Region,1
3,3,N'Djamena,2
4,4,Kigali,3


In [21]:
# merge user_region with country_df to get country name
user_country = user_region.merge(country_df, on="CountryId", how="left")

In [22]:
user_country.head()

,UserId,ContenentId,RegionId_x,CountryId,CityId,Contenent,Region,ContentId,Country,RegionId_y
0,14,5,20,155,220.0,Europe,Southern Europe,5,Portugal,20
1,16,3,14,101,3098.0,Asia,South East Asia,3,Indonesia,14
2,20,4,15,109,4303.0,Australia & Oceania,Australia,4,Australia,15
3,23,1,4,22,154.0,Africa,Southern Africa,1,South Africa,4
4,25,3,14,101,3098.0,Asia,South East Asia,3,Indonesia,14


In [28]:
# merge user_region with city_df to get city name
user_city = user_country.merge(city_df, on="CityId", how="left")

In [29]:
user_city.head()

,UserId,ContenentId,RegionId_x,CountryId_x,CityId,Contenent,Region,ContentId,Country,RegionId_y,CityName,CountryId_y
0,14,5,20,155,220.0,Europe,Southern Europe,5,Portugal,20,Lagos,27.0
1,16,3,14,101,3098.0,Asia,South East Asia,3,Indonesia,14,Jakarta,101.0
2,20,4,15,109,4303.0,Australia & Oceania,Australia,4,Australia,15,Gold Coast,109.0
3,23,1,4,22,154.0,Africa,Southern Africa,1,South Africa,4,Meyerton,22.0
4,25,3,14,101,3098.0,Asia,South East Asia,3,Indonesia,14,Jakarta,101.0


In [32]:
# Merge transaction with items to get item details
transaction_item = transaction_df.merge(item_df, on="AttractionId", how="left")

In [37]:
transaction_item_mode = transaction_item.merge(mode_df, left_on="VisitMode",right_on="VisitModeId", how="left")

In [40]:
# merge transaction item mode with type_df to get type details
transaction_item_mode_type = transaction_item_mode.merge(type_df, on="AttractionTypeId", how="left")

In [43]:
# Merge user information to activities data to get final dataframe
df_final = user_city.merge(transaction_item_mode_type, on="UserId", how="left")

In [46]:
df_final.to_csv('tourism_data_final.csv', index=False)

# Data Cleaning

- Remove unused columns
- Rename columns
- Format values

In [47]:
# Versioning my raw data
df_dirty = df_final.copy()

In [48]:
df_dirty.head()

,UserId,ContenentId,RegionId_x,CountryId_x,CityId,Contenent,Region,ContentId,Country,RegionId_y,...,VisitMode_x,AttractionId,Rating,AttractionCityId,AttractionTypeId,Attraction,AttractionAddress,VisitModeId,VisitMode_y,AttractionType
0,14,5,20,155,220.0,Europe,Southern Europe,5,Portugal,20,...,4,640,4,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",4,Friends,Nature & Wildlife Areas
1,14,5,20,155,220.0,Europe,Southern Europe,5,Portugal,20,...,4,748,5,1,72,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",4,Friends,Points of Interest & Landmarks
2,14,5,20,155,220.0,Europe,Southern Europe,5,Portugal,20,...,4,748,5,1,72,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",4,Friends,Points of Interest & Landmarks
3,16,3,14,101,3098.0,Asia,South East Asia,3,Indonesia,14,...,3,640,5,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",3,Family,Nature & Wildlife Areas
4,16,3,14,101,3098.0,Asia,South East Asia,3,Indonesia,14,...,4,640,5,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",4,Friends,Nature & Wildlife Areas


In [49]:
df_dirty.columns

Index(['UserId', 'ContenentId', 'RegionId_x', 'CountryId_x', 'CityId',
       'Contenent', 'Region', 'ContentId', 'Country', 'RegionId_y', 'CityName',
       'CountryId_y', 'TransactionId', 'VisitYear', 'VisitMonth',
       'VisitMode_x', 'AttractionId', 'Rating', 'AttractionCityId',
       'AttractionTypeId', 'Attraction', 'AttractionAddress', 'VisitModeId',
       'VisitMode_y', 'AttractionType'],
      dtype='object')

In [50]:
main_columns = ['UserId','Contenent', 'Region','Country', 'CityName','VisitYear', 'VisitMonth',
     'Rating','Attraction', 'AttractionAddress','VisitMode_y', 'AttractionType']

In [51]:
df_cut = df_dirty[main_columns]

In [59]:
rename_cols = {
    'Contenent': 'Continent',
    'VisitMode_y': 'VisitMode'
}

In [61]:
df_cut.rename(columns=rename_cols, inplace=True)

/var/folders/nh/kprmp5xn57l72mppyplsnj100000gq/T/ipykernel_16711/1812838551.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut.rename(columns=rename_cols, inplace=True)


In [62]:
df_cut.head()

,UserId,Continent,Region,Country,CityName,VisitYear,VisitMonth,Rating,Attraction,AttractionAddress,VisitMode,AttractionType
0,14,Europe,Southern Europe,Portugal,Lagos,2018,12,4,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Friends,Nature & Wildlife Areas
1,14,Europe,Southern Europe,Portugal,Lagos,2018,12,5,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Friends,Points of Interest & Landmarks
2,14,Europe,Southern Europe,Portugal,Lagos,2018,12,5,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Friends,Points of Interest & Landmarks
3,16,Asia,South East Asia,Indonesia,Jakarta,2018,4,5,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Family,Nature & Wildlife Areas
4,16,Asia,South East Asia,Indonesia,Jakarta,2017,12,5,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Friends,Nature & Wildlife Areas


In [63]:
df_cut.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52930 entries, 0 to 52929
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   UserId             52930 non-null  int64 
 1   Continent          52930 non-null  object
 2   Region             52930 non-null  object
 3   Country            52930 non-null  object
 4   CityName           52922 non-null  object
 5   VisitYear          52930 non-null  int64 
 6   VisitMonth         52930 non-null  int64 
 7   Rating             52930 non-null  int64 
 8   Attraction         52930 non-null  object
 9   AttractionAddress  52930 non-null  object
 10  VisitMode          52930 non-null  object
 11  AttractionType     52930 non-null  object
dtypes: int64(4), object(8)
memory usage: 5.2+ MB


In [64]:
df_cut.to_csv('tourism_data_cleaned.csv', index=False)